#### Proccès general:
- Primer el que fem és un encoding del text. i tenim uns tokens_ids.
    - De representació de text a representació manipulable per l'ordinador. 
    - Durant l'entrenament generarem un vocabulari amb el corpus que li hem passat.
    - El vocabulari servirà per fer l'encoding. 
    - Entrenament, és construir el vocabulari i trobem els encodings. 
- Dos parts: Generar un encoding (fer els tokens).
- Després necessitem un embeding per representar les relacions semantiques, gramaticals, etc de les paraules. 
- Encoding és convertir les paraules en numeros. Convertir en tokens i després amb numeros. 
- Hem de posar aquests numeros en un embedding.
- ---
- Teniem un input, el passem a tokens, els hi donem un id, construim el vocabulari amb token_id i fem un embeding per tenir vectors. 
- L'output és el vector de sortida de la xarxa neuronal de tots els tokens inicials com a input. 
- Després hem de fer el proccès invers per decodificar-lo. que serà un token_id, per trobar el string.
- La reprojecció ens donarà el punt que està més a prop.
- La reprojecció diu de totes les paraules que estàn en el vocabulari. quines estàn més aprop o tenen més similitud. 
- ---
- Donat un input calcula la seguent paraula, el vector de la reprojecció dona el vector de la paraula optima, per saber-ho, mirem la similitud d'aquest punt per tots els altres. Anar al id del vocabulari que es fa servir per relacionar-ho amb les paraules.
- El vocabulari són totes les paraules que hi ha en el corpus d'entrenament.
- ---
- Per generar les respostes de fins a 10 caracters i fa un bucle. 
- Li passem el prompt, calcula la paraula, el concatena amb el prompt i és el següent input del bucle. 

- parameters: com es la nn un cop entrenada. parametres són pesos.
- hyperparametres: la arquitectura determina un numero de hyperparametres.

#### hyperparametres
- llargada del input
- dimensions del embedding
- n tokens sortida (output lenght)

#### parametres del model
- els parametres de la nn que fa l'embedding

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
import tensorflow as tf

2024-05-22 16:49:52.901461: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import gpt2

In [3]:
from utils import load_encoder_hparams_and_params

### load model: 
- ***encoder***
- ***hyperparameters***
- ***parameters***

In [4]:
model_size = '124M'
n_tokens_to_generate = 40
encoder, hparams, params = load_encoder_hparams_and_params(model_size, 'models')

### hyperparameters

In [5]:
hparams

{'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 768, 'n_head': 12, 'n_layer': 12}

In [6]:
_ = '''
{
  "n_vocab": 50257, # number of tokens in our vocabulary
  "n_ctx": 1024, # maximum possible sequence length of the input (input length)
  "n_embd": 768, # embedding dimension (determines the "width" of the network) (dimensions de l'espai, coordenades dels vectors, numero de neurones que tindrà el head)
  "n_head": 12, # number of attention heads (n_embd must be divisible by n_head)
  "n_layer": 12 # number of layers (determines the "depth" of the network)
}
'''

 ### Main loop

#### input encoding (tokenizer)

In [7]:
prompt = 'what is your favorite musical band?'

- El encoder és el vocabulari més el tokenitzador. De paraula a token i de token a id.
- Que sigui un numero és gracies al diccionari.
- Agafa el input i mira quins tokens hi ha i va al vocabulari (diccionari) i pilla el id del token
- pot ser que musical siguin dos tokens.

In [8]:
input_ids = encoder.encode(prompt)
input_ids

[10919, 318, 534, 4004, 10530, 4097, 30]

In [ ]:
encoder.decode(input_ids)
# what is your favorite musical band?

#### output generation
- rep input, parametres i nº tokens a generar.
- fa looper nºtokens
    - generem una sortida en base el input actual
    - mirem la sortida, els logits,
    - similitud de qualsevol token del diccionari amb la sortida optima
    - greedy, ens quedem el logit més activat
    - això és la id que hem de venir a buscar al diccionari vocabulari.
    - la id la enganxem a els inputs

In [9]:
def _generate(inputs, params, n_head, n_tokens_to_generate):

    for _ in range(n_tokens_to_generate):
        
        logits = gpt2.gpt2(inputs, **params, n_head=n_head)  # model forward pass
        next_id = gpt2.np.argmax(logits[-1])                 # greedy sampling
        inputs.append(int(next_id))                          # append prediction to input

    return inputs # Sería la pregunta més la responsta perque els hem anant concatenant

In [10]:
n_tokens_to_generate = 11
output_ids = _generate(input_ids, params, hparams['n_head'], n_tokens_to_generate)
output_ids[:-n_tokens_to_generate], output_ids[-n_tokens_to_generate:]

([10919, 318, 534, 4004, 10530, 4097, 30],
 [198, 198, 40, 1842, 262, 2647, 286, 262, 44249, 5542, 13])

#### output decoding

In [11]:
print(encoder.decode(output_ids))

what is your favorite musical band?

I love the music of the Grateful Dead.


- 198 és l'intro. 
- 1 paraula no sempre es convertirà en un token, dependrà del entrenament. 
- li passem un text i genera una paraula i una altre i va fent. i respon al input que li hem donat.

### Autoregressive generation loop
- Ho fem tot de cop

In [12]:
def _generate_decode(inputs, params, n_head, n_tokens_to_generate):

    print(inputs)
    print(encoder.decode(inputs))
    print()
    
    outputs = []
    for _ in range(n_tokens_to_generate):
        
        logits = gpt2.gpt2(inputs, **params, n_head=n_head)  # model forward pass
        next_id = gpt2.np.argmax(logits[-1])                 # greedy sampling
        
        inputs.append(int(next_id))                           # append prediction to input
        outputs.append(int(next_id))                          # append prediction to output

        # current input/output
        print(inputs)
        print(encoder.decode(outputs))

    return outputs


In [13]:
input_ids = encoder.encode(prompt)
input_ids

[10919, 318, 534, 4004, 10530, 4097, 30]

In [25]:
n_tokens_to_generate = 1
output_ids = _generate_decode(input_ids, params, hparams['n_head'], n_tokens_to_generate)

[10919, 318, 534, 4004, 10530, 4097, 30, 198, 198, 40, 1842, 262, 2647, 286, 262, 44249, 5542, 13]
what is your favorite musical band?

I love the music of the Grateful Dead.

[10919, 318, 534, 4004, 10530, 4097, 30, 198, 198, 40, 1842, 262, 2647, 286, 262, 44249, 5542, 13, 314]
 I


- Això és l'esquema bàsic.
- Hi ha més coses com sortides de diferent llargada, diferents respostes en funció de la temperatura, etc.

#### Multiheading
- Vàries vegades les mateixes capes (en paral·lel) i dona un altre vector. 
- En podem tenir 10/20/100
- Cada estructura, genera un embedding diferent.
- Cada un detecta diferents matizos diferents de les paraules.
- Multipliquem el numero de parametres que te el sistema, però el que fem és que quan generem una sortida, tenim més possibilitats.
- Cada vector final (output) en tenim un per cada head.
- Fem un entrenament per totes les nn i es van ajustant els pesos i al final cada head surt diferent.
- Pot ser perque com que inicialitzem random, i anem ajustant, anem a minims locals diferents.
- ---
- un embedding que separa semanticament, un altre relacions sintactiques. Pretenem que la proximitat i similitud que expresen les distancies en els embeddings, capturi aspectes diferents de les relacions amb les paraules.
- El mateix que feiem amb les xarxes convolucionals, que posavem diferents filtres (canals).
- convinem tots els vectors finals, per reprojectar.
- tots els embeddings tenen la mateixa dimensio.
- Aquí es on hi ha l'aritmetica de rey - home = reina. Un embedding sería el que representa el genere.
- ---
- Un altre embedding que hi ha en aquest model i el separa és el word position embedding. Com s'ha generat, no ho sabem.
    - no es el mateix una paraula al principi que al final. L'ordre importa.

#### Característica dels models
- Podem fer anar tot això sense model. 
- Es multiplicar els ids per una matriu que representa tot el que ha passat a la nn. 